In [31]:
import pickle
from pylatex import Document, Section, Enumerate
from pylatex.utils import italic, bold, dumps_list, NoEscape
import itertools
import random

In [32]:
def read_data_iterator(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        for i in itertools.count():
            try:
                yield pickle.load(f)
            except EOFError:
                return

In [40]:
#data = list(read_data_iterator('../extractions.pickle'))
data = [e for i, e in zip(range(200), read_data_iterator('gw_extractions.pickle'))]
#random.seed('event_chain_1')
random.seed('event_chain_gw')
random.shuffle(data)
data[0]['entity']

data = [d for d in data if len(' '.join(d['sentences']).split()) < 150]
len(data)

162

In [41]:
def create_pdf(data, pdf_name, use_triples=False, answer_key=False):
    text_key = ('triples' if use_triples else 'sentences')
    doc = Document()
    with doc.create(Enumerate()) as question_enum:
        for i, seq in zip(range(20), data):
            if use_triples:
                question_enum.add_item(
                    dumps_list((
                       NoEscape("(%s)" % dumps_list((italic(seq['entity']) if t is None else t for t in triple), token=', ')) 
                       for triple in seq['triples']
                    ), 
                        token=', ')
                )
            else:
                question_enum.add_item(' '.join([s.strip() for s in seq['sentences']]))
            with doc.create(Enumerate(enumeration_symbol=r"\alph*)")) as candidate_enum:
                for i, candidate  in enumerate(seq['candidates']):
                    lat_str = dumps_list((italic(seq['entity']) if c is None else c for c in candidate), escape=False, token=', ')
                    lat_str = NoEscape("(%s)"%lat_str)
                    if answer_key and i == seq['correct']:
                        lat_str = bold(lat_str)
                    candidate_enum.add_item(lat_str)

    doc.generate_pdf(pdf_name, silent=True, clean_tex=True)
    
create_pdf(data[:20], 'event_chain_gw_ak', answer_key=True)
create_pdf(data[:20], 'event_chain_gw', answer_key=False)
create_pdf(data[:20], 'event_chain_gw_triple', use_triples=True, answer_key=False)